In [1]:
import json
import os
import pickle
import random
import sys
import time

import cv2
import ibm_boto3
import numpy as np
import pandas as pd
from ibm_botocore.client import Config
from tqdm import tqdm


In [2]:
# cred_path = "/home/alex/Documents/MIDS/w210/FinalProject/w210-credentials.json"
cred_path = "/tmp/w210-credentials.json"
with open(cred_path, "r") as f:
    creds = json.load(f)

auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

# Store relevant details for interacting with IBM COS store and uploading data
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=creds['apikey'],
                         ibm_service_instance_id=creds['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

In [3]:
# FIXME: need to figure out how to install aspera for massive download speed
# boost. Surprisingly, the docs suck
#
# from ibm_s3transfer.aspera.manager import AsperaTransferManager
# cos = ibm_boto3.client('s3',
#                          ibm_api_key_id=creds['apikey'],
#                          ibm_service_instance_id=creds['resource_instance_id'],
#                          ibm_auth_endpoint=auth_endpoint,
#                          config=Config(signature_version='oauth'),
#                          endpoint_url=service_endpoint)
# transfer_manager = AsperaTransferManager(cos)

In [4]:
bucket = cos.Bucket('w210-finalproject')

In [5]:
files = list(bucket.objects.all())

/root/bittah-ninja/w210/lib/python3.6/site-packages/ibm_botocore/vendored/requests/api.py:67: DeprecationWarning: You are using the post() function from 'ibm_botocore.vendored.requests'.  This is not a public API in ibm_botocore and will be removed in the future. Additionally, this version of requests is out of date.  We recommend you install the requests package, 'import requests' directly, and use the requests.post() function instead.
  DeprecationWarning


In [6]:
savepath = '/tmp/vids'
os.makedirs(savepath, exist_ok=True)
filelist = os.listdir(savepath)

In [7]:
# savepath = "/home/alex/Documents/MIDS/w210/FinalProject/tmp"
savepath = '/tmp/vids'
os.makedirs(savepath, exist_ok=True)
filelist = os.listdir(savepath)
for file in tqdm(files):
    ext = file.key.split('.')[-1]
    if 'mp4' not in ext and 'avi' not in ext:
        continue
    filename = file.key.split('/')[-1]
    if filename in filelist:
        continue
    else:
        filename = os.path.join(savepath, filename)
        bucket.download_file(file.key, filename)


100%|██████████| 808/808 [00:00<00:00, 159602.41it/s]


In [15]:
vids = os.listdir(savepath)
cap = cv2.VideoCapture(os.path.join(savepath, random.sample(vids,1)[0]))

In [ ]:
while(cap.isOpened()):
    _, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [13]:
frame.shape

(720, 1280, 3)

In [14]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray.shape

(720, 1280)

In [ ]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.imshow('vid', gray)